In [1]:
def one_hot_encode(y):
    values = np.unique(y).size
    encoded_outputs = []

    for i in range(len(y)):
        new_output = [0] * values
        new_output[y[i]] = 1
        encoded_outputs.append(new_output)

    return np.array(encoded_outputs)

In [2]:
import numpy as np
import math
import tensorflow

(x_train, y_train), (x_test, y_test) = tensorflow.keras.datasets.mnist.load_data()

x_train = x_train.reshape((60000, 784))
x_test = x_test.reshape((10000, 784))

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.T
x_test = x_test.T # Transposed to match the standard dimensions... The training/testing examples should be column vectors.

y_train_original = y_train
y_test_original = y_test

y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)

y_train = y_train.T
y_test = y_test.T

2025-10-07 00:49:19.007091: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-07 00:49:19.007794: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 00:49:19.117243: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-07 00:49:21.380409: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [3]:
class DFN:
    def __init__(self):
        
        self.weights = []
        self.weights.append(0)
        # Initialize using He initialization
        self.weights.append(np.random.randn(128, 784) * np.sqrt(2. / 784)) 
        self.weights.append(np.random.randn(16, 128) * np.sqrt(2. / 128)) 
        self.weights.append(np.random.randn(10, 16) * np.sqrt(2. / 16))


        self.biases = []
        self.biases.append(0)
        self.biases.append(np.zeros((128, 1)))
        self.biases.append(np.zeros((16, 1)))
        self.biases.append(np.zeros((10, 1)))

    def sigmoid(self, x): # unused
        return 1 / (1 + np.exp(- x))

    def sigmoid_derivative(self, x): #unused
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def RELU(self, x):
        return np.maximum(x, 0)

    def RELU_derivative(self, x):
        return (x > 0).astype(int)

    def softmax(self, x): 
        # shift x to be centered around zero for numerical stability, may give inf otherwise
        shifted_x = x - np.max(x, axis=0, keepdims=True)
        exps = np.exp(shifted_x)
        return exps / np.sum(exps, axis=0, keepdims=True)

    def softmax_derivative(self):
        pass # implemented in the code.

    def forward_prop(self, input): # note: input has to be a strict column vector here.
        a = []
        z = []
        
        a.append(input)
        z.append([0])
        z_temp = (self.weights[1] @ a[0]) + self.biases[1] # @ is pretty much just matrix multiplication.
        a_temp = self.RELU(z_temp)

        z.append(z_temp)
        a.append(a_temp)

        z_temp = (self.weights[2] @ a[1]) + self.biases[2]
        a_temp = self.RELU(z_temp)

        z.append(z_temp)
        a.append(a_temp)

        z_temp = (self.weights[3] @ a[2]) + self.biases[3]
        a_temp = self.softmax(z_temp)

        z.append(z_temp)
        a.append(a_temp)

        return z, a

    def back_prop(self, z, a, y, alpha): # dX is the derivative of loss wrt X.
        dZ3 = a[3] - y # It's a neat calculus trick that skips the jacobian calculation for softmax derivative altogether.
        dW3 = dZ3 @ a[2].T
        dB3 = dZ3 # Calculating and chaining up the derivatives like we did in the last code...
        
        dA2 = self.weights[3].T @ dZ3  # column vector
        dZ2 = dA2 * self.RELU_derivative(z[2]) # column vector
        dW2 = dZ2 @ a[1].T
        dB2 = dZ2

        dA1 = self.weights[2].T @ dZ2 
        dZ1 = dA1 * self.RELU_derivative(z[1])
        dW1 = dZ1 @ a[0].T
        dB1 = dZ1

        # update everything.

        self.weights[3] -= alpha * dW3
        self.biases[3] -= alpha * dB3

        self.weights[2] -= alpha * dW2
        self.biases[2] -= alpha * dB2

        self.weights[1] -= alpha * dW1
        self.biases[1] -= alpha * dB1

    def train(self, x_train, y_train, y_train_original, learning_rate = 0.01, epochs = 20):
        samples = x_train.shape[1]

        for epoch in range(epochs):
            print(f"Epoch - {epoch}")
            epoch_loss = 0
            
            for i in range(samples):
                x = x_train[:, i].reshape(-1, 1)
                y = y_train[:, i].reshape(-1, 1)
                # Now we have x and y, single examples as column vectors.
    
                z, a = self.forward_prop(x)
                y_class_index = y_train_original[i]
                epsilon = 1e-9 # or else log(0) gives inf.
                loss = - np.log(a[3][y_class_index, 0] + epsilon)
                self.back_prop(z, a, y, alpha = learning_rate)
                epoch_loss += loss
                
            # calculate epoch loss averaged over epoch
            epoch_loss /= samples
            print(f"Epoch loss = {epoch_loss}\n")

        print("Training complete.")

In [4]:
model = DFN()

model.train(x_train, y_train, y_train_original)

Epoch - 0
Epoch loss = 0.22478799041488176

Epoch - 1
Epoch loss = 0.10830480374033896

Epoch - 2
Epoch loss = 0.07860321085063143

Epoch - 3
Epoch loss = 0.06294502715091953

Epoch - 4
Epoch loss = 0.053277201644223275

Epoch - 5
Epoch loss = 0.04763272127417491

Epoch - 6
Epoch loss = 0.04345994598034939

Epoch - 7
Epoch loss = 0.038325217962686606

Epoch - 8
Epoch loss = 0.037497168224951825

Epoch - 9
Epoch loss = 0.03248101209132064

Epoch - 10
Epoch loss = 0.0282300614045651

Epoch - 11
Epoch loss = 0.028483056005214953

Epoch - 12
Epoch loss = 0.02569521260641412

Epoch - 13
Epoch loss = 0.026929421590358795

Epoch - 14
Epoch loss = 0.024921746197188444

Epoch - 15
Epoch loss = 0.021780927763742363

Epoch - 16
Epoch loss = 0.02194918497742459

Epoch - 17
Epoch loss = 0.024217969334076602

Epoch - 18
Epoch loss = 0.01935406615656663

Epoch - 19
Epoch loss = 0.022683713267969714

Training complete.


In [47]:
# Model testing

def evaluate_model(model, x_test, y_test_original):
    print("\nStarting model evaluation on the test set...")
    
    correct_predictions = 0
    num_samples = x_test.shape[1]

    for i in range(num_samples):
        image = x_test[:, i].reshape(-1, 1)
        true_label = y_test_original[i]

        _, a = model.forward_prop(image)
        predicted_label = np.argmax(a[3])

        if predicted_label == true_label:
            correct_predictions += 1
            
    accuracy = (correct_predictions / num_samples) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Correctly classified {correct_predictions} out of {num_samples} samples.")

In [55]:
evaluate_model(model, x_test, y_test_original) # It generally performs better than the previous model. Let's see if we can make it better...


Starting model evaluation on the test set...
Test Accuracy: 97.34%
Correctly classified 9734 out of 10000 samples.
